### Non-collocated control of two point masses connected through a spring:



I am modelling a very simple system with two point masses $m_0, m_1$. 
\
These each have a degree of freedom, $\theta_0$ and $\theta_1$, which are defined as the distance from the origin in 1D. 
\
The points are connected through a spring of stiffness k and neutral length 1.
See image below:

![alt text](blocks.png "Title")


This leads to the following Euler-Lagrange formulation:

Kinetic energy:

$T = \frac{1}{2} m_0 \dot{\theta}_0^2 + \frac{1}{2} m_1 \dot{\theta}_1^2$

Potential energy:

$V = \frac{1}{2} k ((\theta_1 - 1) - \theta_0)^2$

Lagrangian L = T - V:

$L = \frac{1}{2} m_0 \dot{\theta}_0^2 + \frac{1}{2} m_1 \dot{\theta}_1^2 - \frac{1}{2} k ((\theta_1 - 1) - \theta_0)^2$

$\frac{d}{dt}(\frac{\partial L}{\partial \dot{\theta_i}}) - \frac{\partial L}{\partial \theta_i} = 0$

$\frac{d}{dt}(\frac{\partial L}{\partial \dot{\theta_0}}) = m_i \ddot{\theta_0}$, &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\frac{\partial L}{\partial \theta_0} = - k ((\theta_1 - 1) - \theta_0)$

$\frac{d}{dt}(\frac{\partial L}{\partial \dot{\theta_1}}) = m_i \ddot{\theta_1}$, &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\frac{\partial L}{\partial \theta_1} = k ((\theta_1 - 1) - \theta_0)$

### Matrix Form

The Euler-Lagrange equations in matrix form is:

$\mathbf{M} \ddot{\theta} + \mathbf{G}(\theta) = \mathbf{A} u$

With mass matrix:

$\mathbf{M} = \begin{bmatrix} m_0 & 0 \\\ 0 & m_1 \end{bmatrix}$

potential matrix:

$\mathbf{G}(\theta) = \begin{bmatrix} \phantom{-.} k (\theta_0 - (\theta_1 - 1))\\\ -k (\theta_0 - (\theta_1 - 1))\end{bmatrix}$ 

and input matrix:

$A = \begin{bmatrix} 1 \\\ 0 \end{bmatrix}$

Resulting in the full description:

$ \begin{bmatrix} m_0 & 0 \\\ 0 & m_1 \end{bmatrix} + \begin{bmatrix} \phantom{-.} k (\theta_0 - (\theta_1 - 1))\\\ -k (\theta_0 - (\theta_1 - 1))\end{bmatrix} = \begin{bmatrix} 1 \\\ 0 \end{bmatrix}$

### Non-collocated coordinate

Select as first coordinate the unactuated DoF, and following coordinates its derivatives:

$y = \theta_1$

$\dot{y} = \dot{\theta}_1$

$\ddot{y} = \ddot{\theta}_1 = \frac{k}{m_1}(\theta_0 - (\theta_1 - 1))$

$\dddot{y} = \frac{k}{m_1}(\dot{\theta}_0 - \dot{\theta}_1 )$

$y^{iv} = \frac{k}{m_1}(\ddot{\theta}_0 - \ddot{\theta}_1 ) = \frac{k}{m_1} (\frac{1}{m_0}(u - k(-\theta_0 + (\theta_1 - 1))) - \frac{k}{m_1}(\theta_0 - (\theta_1 - 1)))$

### Normal form 

This system has a normal form of:

$y^{iv} = \alpha(Y) + \beta(Y) u$

Where

$ \alpha(Y) = - \frac{k}{m_1} (\frac{1}{m_0}(k(-\theta_0 + (\theta_1 - 1))) + \frac{k}{m_1}(\theta_0 - (\theta_1 - 1)))$

and

$ \beta(Y) = \frac{k}{m_0 m_1}u$

### Control Law
Define a desired state:

$\bar{Y} = f(\bar{\theta})$

And control law:

$v = K(\bar{Y} - Y)$

Use this as a desired $y^{iv}$ to obtain the required control input u:

$u = \frac{1}{\beta(Y)} (-\alpha(Y) + v)$

In [ ]:
import torch
import matplotlib.pyplot as plt

# Basic parameters of the simulation
rp_b = {
    "m0": 3.,
    "m1": 2.,
    "k": 1.
}

In [ ]:
def dynamical_matrices(rp_b, th):

    """
    M and G follow from Euler-Lagrange derivation.
    """

    M = torch.tensor([[rp_b["m0"], 0],
                      [0, rp_b["m1"]]])

    G = torch.tensor([[ rp_b["k"]*(th[0] - (th[1] - 1))],
                      [-rp_b["k"]*(th[0] - (th[1] - 1))]])
    
    return M, G

def input_matrix():

    """
    Input matrix is simply [I, 0]^T.
    """

    A = torch.tensor([[1],
                      [0]])
    
    return A

In [ ]:
def calculate_Y(rp_b, th, th_d):

    """
    Select unactuated DoF as output and derive.
    I checked the substitutions in Wolfram Mathematica and also by hand. 
    (But of course I may have made a mistake)
    """

    y_0 = th[1]
    y_1 = th_d[1]
    y_2 = rp_b["k"]/rp_b["m1"] * (th[0] - (th[1] - 1))
    y_3 = rp_b["k"]/rp_b["m1"] * (th_d[0] - th_d[1])
    
    Y = torch.tensor([[y_0],
                      [y_1],
                      [y_2],
                      [y_3]])
    
    return Y

def calculate_alpha_beta(rp_b, th, th_d):

    """
    This function calculates alpha and beta in the equation:
    y(iv) = alpha + beta * u
    """

    alpha = rp_b["k"]**2/rp_b["m1"] * (1/rp_b["m0"] * ((th[1] - 1) - th[0]) + 1/rp_b["m1"] * ((th[1] - 1) - th[0]))

    
    beta = rp_b["k"]/(rp_b["m0"]*rp_b["m1"])
    return alpha, beta

def calculate_v(Y, Y_des, K):

    """
    Calculate the virtual control input based on gains K and error.
    """

    v = K @ (Y_des - Y)
    return v

def calculate_u(alpha, beta, v):

    """
    Obtain final control input.
    """

    u = 1/beta * (-alpha + v)
    return u

In [ ]:
def plot_double_vs_time(double_list, dt, ylabel, title):
    time = torch.arange(0, double_list.shape[0] * dt, dt).numpy()
    double_np = double_list.numpy()

    plt.figure(figsize=(7, 3))
    plt.plot(time, double_np[:, 0], label="θ̇0")
    plt.plot(time, double_np[:, 1], label="θ̇1")
    plt.xlabel("Time (s)")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_single_vs_time(single_list, dt):
    time = torch.arange(0, single_list.size(0) * dt, dt).numpy()
    single_np = single_list.numpy()

    plt.figure(figsize=(7, 3))
    plt.plot(time, single_np, label="u")
    plt.xlabel("Time (s)")
    plt.ylabel("Control Input (N)")
    plt.title("Control Input vs Time")
    plt.legend()
    plt.grid(True)
    #plt.yscale("log")
    plt.tight_layout()
    plt.show()

def plot_quad_vs_time(quad_list, dt, ylabel, title):
    time = torch.arange(0, quad_list.shape[0] * dt, dt).numpy()
    quad_np = quad_list.numpy()

    plt.figure(figsize=(7, 3))
    plt.plot(time, quad_np[:, 0], label="Y")
    plt.plot(time, quad_np[:, 1], label="Y'")
    plt.plot(time, quad_np[:, 2], label="Y''")
    plt.plot(time, quad_np[:, 3], label="Y'''")
    plt.xlabel("Time (s)")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
K = torch.tensor([10., 100., 100., 100.])
#K = K/70

# This starts the system without any spring force or velocity.
th_start = torch.tensor([[0., 1.]])
th_d_start = torch.tensor([[0.0, 0.0]])

# The desired state is also with th1 - th0 = 1, so this should be achievable.
th_des = torch.tensor([[3., 5.]])
th_d_des = torch.tensor([[0., 0.]])

Y_des = calculate_Y(rp_b, th_des.squeeze(0), th_d_des.squeeze(0))
Y_test = calculate_Y(rp_b, th_start.squeeze(0), th_d_start.squeeze(0))
alpha_test, beta_test = calculate_alpha_beta(rp_b, th_start.squeeze(0), th_d_start.squeeze(0))
v_test = calculate_v(Y_test, Y_des, K)
u_test = calculate_u(alpha_test, beta_test, v_test)

print(Y_des)


In [ ]:
t_start = 0.
t_end = 40
dt = 0.01

th = th_start
th_d = th_d_start

th_list = torch.empty((0, 2))
th_d_list = torch.empty((0, 2))
u_list = torch.empty((0))
Y_list = torch.empty((0, 4))
Y_des_list = torch.empty((0, 4))

for t in torch.arange(t_start, t_end, dt):
    #print("t:", t.item())
    
    M, G = dynamical_matrices(rp_b, th.squeeze(0))
    A = input_matrix()

    Y = calculate_Y(rp_b, th.squeeze(0), th_d.squeeze(0))
    alpha, beta = calculate_alpha_beta(rp_b, th.squeeze(0), th_d.squeeze(0))
    v = calculate_v(Y, Y_des, K)
    u = calculate_u(alpha, beta, v)
    #gravity_compesnation = 
    #u = u + tau_e (gravity compensation)

    th_dd = torch.pinverse(M) @ (A * u - G).squeeze(1)
    th_d = th_d + th_dd * dt
    th = th + th_d * dt
    
    th_list = torch.concatenate((th_list, th))
    th_d_list = torch.concatenate((th_d_list, th_d))
    u_list = torch.concatenate((u_list, u))
    Y_list = torch.concatenate((Y_list, Y.T))
    Y_des_list = torch.concatenate((Y_des_list, Y_des.T))



In [ ]:
plot_double_vs_time(th_list, dt, "Position (m)", "Block Position vs Time")
plot_double_vs_time(th_d_list, dt, "Velocity (m/s)", "Block Velocities vs Time")
plot_single_vs_time(u_list, dt)
plot_quad_vs_time(Y_list, dt, "Y", "Y vs Time")
plot_quad_vs_time(Y_des_list, dt, "Y_des", "Y_des vs Time")
